In [40]:
import time
import datetime 
import ipaddress
import random
import nmap
import masscan
import os
import pandas as pd
from tabulate import tabulate
from collections import OrderedDict


top_20_ports = ['80','23','443','21','22','25','3389','110','445','139','143','53','135','3306','8080','1723','111','995','993','5900']
ranges = ['10.10.14.0/23','192.168.3.0/24','192.168.20.0/30']# '172.16.0.0/16']#,'10.0.0.0/8'] #,'10.10.10.0/24','192.168.2.0/24','172.16.0.0/12']
tracking = {}
for range in ranges:
    tracking[range]={'uphost_count':0,
                     'responsive':{}}

class IPer():
    def __init__(self, ip_range):
        self.ip_range = ipaddress.IPv4Network('192.168.2.0/24')
    
    def get_samples(host_ips,percentage):
        
        hosts = host_ips
        print('TOTAL Possible IPs: {}'.format(len(hosts)))
        ten_percent = len(hosts)*percentage
        choosen1 = random.sample(hosts,int(ten_percent))
        after_choosen1 = list(set(hosts)-set(choosen1))
        choosen2 = random.sample(after_choosen1,int(ten_percent))

        after_choosen2 = list(set(after_choosen1)-set(choosen2))
        choosen3 = random.sample(after_choosen2,(int(ten_percent)))
        
        choosen1_strings = []
        for ip in choosen1:
            choosen1_strings.append(str(ip))
        
        choosen2_strings = []
        for ip in choosen2:
            choosen2_strings.append(str(ip))
            
        choosen3_strings = []
        for ip in choosen3:
            choosen3_strings.append(str(ip))
                                
        return choosen1_strings,choosen2_strings,choosen3_strings
    
    def get_my_local_ip():
        os.system("ip -br address | grep eth0  | cut -d 'P' -f 2 | cut -d ' ' -f 14 | cut -d '/' -f 1 > file.txt")
        with open("file.txt",'r') as f:
            ip = f.read()
            ip_strip = ip.strip()
            return ip_strip

    
def nmap_top_hundred(the_range,host_ips):
    scanner_ip = IPer.get_my_local_ip()
    print("[*]RUNNING NMAP TOP 100 PORTS -- SAMPLE FROM {}[*]".format(the_range))
    nm = nmap.PortScanner()
    if the_class == "C":
        #scan entire range
        scan_data =nm.scan(hosts=the_range, arguments='-Pn --open -T5 -n --exclude {} --top-ports 100'.format(scanner_ip))
        for each_ip in scan_data['scan'].keys():
            if each_ip not in tracking[the_range]['responsive'].keys():          
                tracking[the_range]['uphost_count'] += 1
                tracking[the_range]['responsive'][each_ip]= list(scan_data['scan'][each_ip]['tcp'].keys())
            else:
                tracking[the_range]['responsive'][each_ip] += list(scan_data['scan'][each_ip]['tcp'].keys())

        
    elif the_class == "B":
        #take 10% sample and pingsweep it
        sample1,sample2,sample3 = IPer.get_samples(host_ips,.1)
        scan_data =nm.scan(hosts=' '.join(sample1), arguments='-Pn --open -T5 -n --exclude {} --top-ports 100'.format(scanner_ip))
        for each_ip in scan_data['scan'].keys():
            if each_ip not in tracking[the_range]['responsive'].keys():          
                tracking[the_range]['uphost_count'] += 1
                tracking[the_range]['responsive'][each_ip]= list(scan_data['scan'][each_ip]['tcp'].keys())
            else:
                tracking[the_range]['responsive'][each_ip] += list(scan_data['scan'][each_ip]['tcp'].keys())
        
    elif the_class == "A":
        sample1,sample2,sample3 = IPer.get_samples(host_ips,.005)

        scan_data =nm.scan(hosts=' '.join(sample1), arguments='-Pn --open -T5 -n --exclude {} --top-ports 100'.format(scanner_ip))
        for each_ip in scan_data['scan'].keys():
            if each_ip not in tracking[the_range]['responsive'].keys():          
                tracking[the_range]['uphost_count'] += 1
                tracking[the_range]['responsive'][each_ip]= list(scan_data['scan'][each_ip]['tcp'].keys())
            else:
                tracking[the_range]['responsive'][each_ip] += list(scan_data['scan'][each_ip]['tcp'].keys())
    '''   
    scan1=''
    scan2=''
    scan3=''
    scanner_ip = IPer.get_my_local_ip()
    nm = nmap.PortScanner()
    print('SCAN #1')
    scan_data =nm.scan(hosts=' '.join(sample1), arguments='-Pn --open -T5 -n --exclude {} --top-ports 100'.format(scanner_ip))
    for each_ip in scan_data['scan'].keys():
        if each_ip not in tracking[the_range]['responsive'].keys():          
            tracking[the_range]['uphost_count'] += 1
            tracking[the_range]['responsive'][each_ip]= list(scan_data['scan'][each_ip]['tcp'].keys())
        else:
            tracking[the_range]['responsive'][each_ip] += list(scan_data['scan'][each_ip]['tcp'].keys())
            #print('{} {}'.format(each_ip, list(scan_data['scan'][each]['tcp'].keys())))
     '''



##################
# TO DO:
#       1. Add checks to determine fastest rate for network allowed
def masscan_ports(the_range,top_20_ports):
        #will use samples
        sample1,sample2,sample3 = IPer.get_samples(the_range,.3)
        samples = [sample1,sample2,sample3]
        ms = masscan.PortScanner()
        
        for sample in samples:
            print(sample)
            if tracking[the_range] == 0:
                ms.scan(' '.join(sample),ports='{}'.format(','.join(top_20_ports)),arguments="--router-mac 66-55-44-33-22-11 -e eth0 --max-rate 30000")
                #place holder return value from historic run on hackthebox.com
                '''
                ms = {ipaddress.ip_address(('10.129.90.245')): ['tcp', [80, 135, 139, 445]],
                      ipaddress.ip_address(('10.129.90.241')): ['tcp', [80]],
                      ipaddress.ip_address(('10.129.90.146')): ['tcp', [22, 80]],
                      ipaddress.ip_address(('10.129.90.220')): ['tcp', [80]],
                      ipaddress.ip_address(('10.129.90.128')): ['tcp', [80]]}
                '''
                the_count = 0
                for ip,ports in ms.items():
                    print("THIS IS IP: {}/tTHIS IS PORTS: {}".format(ip,ports))
                    if len(ports) > 0:
                        the_count += 1
                        tracking[the_range]['uphost_count'] += 1
                        tracking[the_range]['responsive_ips'].append(ip)
                        tracking[the_range]['responsive_ports'].append([ports,'masscan_portscan'])
                
def fping_sweep(the_range,host_ips,the_class):
    print("[*]RUNNING PING SWEEP[*]")
    if the_class == "C":
        #scan entire range
        cmd='fping -4 --addr -r 1 -a -i 1 -g {} 2>/dev/null >> fping_uphosts.txt'.format(the_range)
        out = os.system(cmd)

        
    elif the_class == "B":
        #take 10% sample and pingsweep it
        sample1,sample2,sample3 = IPer.get_samples(host_ips,.1)
        cmd = 'fping -4 --addr -r 1 -a -i 1 {} 2>/dev/null >> fping_uphosts.txt'.format(' '.join(sample1))
        print("SAMPLE IPs FROM RANGE: {}".format(sample1))
        out = os.system(cmd)
        
    elif the_class == "A":
        sample1,sample2,sample3 = IPer.get_samples(host_ips,.005)
        cmd = 'fping -4 --addr -r 1 -a -i 1 {} 2>/dev/null >> fping_uphosts.txt'.format(' '.join(sample1))
        out = os.system(cmd)
    
    # Account results in tracking dict    
    with open('fping_uphosts.txt','r') as f:
        ips = f.readlines()
        if len(ips) > 0:
            scanner_ip = IPer.get_my_local_ip()
            for ip in ips:
                if scanner_ip not in ip:
                    tracking[the_range]['uphost_count'] += 1
                    if ip not in tracking[the_range]['responsive'].keys():
                        tracking[the_range]['responsive'][ip.strip()] = ['ICMP']
                else:
                    pass
    os.system('rm fping_uphosts.txt')
    print('[*]PING SWEEP COMPLETE[*]')
    
    
def unicorn_ports(the_range,top_20_ports,host_ips):
    sample1,sample2,sample3 = IPer.get_samples(host_ips,.1)
    samples = [sample1,sample2,sample3]
    ips = []
    ports = []

    # sudo unicornscan --immediate -mT -R 1 -r100000 -i eth0 192.168.3.0/24 --ports 80,443,22
    if the_class == "A":
        sample1,sample2,sample3 = IPer.get_samples(host_ips,.05)
        cmd='sudo unicornscan -mTS -r200000 -L 2 -i tun0 {} --ports {} >> unicornscan.out'.format(' '.join(sample1),','.join(top_20_ports))
        print('[*]RUNNING UNICORNSCAN -- TOP 20 PORTS on SAMPLE FROM {}[*]'.format(the_range))
        out = os.system(cmd)
    elif the_class == "B":
        sample1,sample2,sample3 = IPer.get_samples(host_ips,.1)
        cmd='sudo unicornscan -mTS -r200000 -L 2 -i tun0 {} --ports {} >> unicornscan.out'.format(' '.join(sample1),','.join(top_20_ports))
        print('[*]RUNNING UNICORNSCAN -- TOP 20 PORTS on SAMPLE FROM {}[*]'.format(the_range))
        out = os.system(cmd)
    else:
        #Class C
        sample1,sample2,sample3 = IPer.get_samples(host_ips,.3)
        cmd='sudo unicornscan -mTS -r200000 -L 2 -i tun0 {} --ports {} >> unicornscan.out'.format(' '.join(sample1),','.join(top_20_ports))
        print('[*]RUNNING UNICORNSCAN -- TOP 20 PORTS on SAMPLE FROM {}[*]'.format(the_range))
        out = os.system(cmd)
    
    unicorn_dict = {}    
    with open('unicornscan.out','r') as f:
        data = f.readlines()
        if len(data) > 0:
            for line in data:
                split_line = line.split('from')
                last = split_line[-1]
                split_last = last.split(' ')
                #print("SPLIT_LAST: {}".format(split_last))
                try:
                    final_ip = split_last[1]
                    split_port_step1 = line.split('[')
                    if len(split_port_step1) > 1:
                        split_port_step2 = split_port_step1[1]
                        split_port_step3 = split_port_step2.split(']')
                        port_number=split_port_step3[0].strip()

                        if unicorn_dict.get(final_ip):
                            if port_number not in unicorn_dict[final_ip]:
                                unicorn_dict[final_ip] += [port_number]
                        else:
                            unicorn_dict[final_ip] = [port_number]
                except:
                    pass

    for ip_,ports_ in unicorn_dict.items():
        if ip_ not in tracking[the_range]['responsive'].keys():
            tracking[the_range]['uphost_count'] += 1
            tracking[the_range]['responsive'][ip_] = ports_
        elif ports_ not in tracking[the_range]['responsive'][ip_]:
            tracking[the_range]['responsive'][ip_] += ports_
    
    os.system('rm unicornscan.out')

        

In [41]:
for range in ranges:
    # First Ping Sweep Entire Ranges if Class C
    host_ips = list(ipaddress.ip_network(range).hosts())
    number_of_hosts = len(host_ips)
    if  number_of_hosts <= 32512:
        print("{} range is C".format(range))
        the_class = "C"
        fping_sweep(range,host_ips, the_class)

        
    # If Class B /16 (65,024) then sample 10% with fping
    elif number_of_hosts  >= 65024 and len(list(ipaddress.ip_network(range).hosts())) < 16777214:
        print("{} range is B".format(range))
        fping_sweep(range,host_ips, the_class)
        
    # If Class A /8 (16,777,214)then sample half a 2.5% with fping
    elif number_of_hosts >= 16777214:
        print("{} range is A".format(range))
        fping_sweep(range,host_ips, the_class)
        
tracking
    

10.10.14.0/23 range is C
[*]RUNNING PING SWEEP[*]
[*]PING SWEEP COMPLETE[*]
192.168.3.0/24 range is C
[*]RUNNING PING SWEEP[*]
[*]PING SWEEP COMPLETE[*]
192.168.20.0/30 range is C
[*]RUNNING PING SWEEP[*]
[*]PING SWEEP COMPLETE[*]


{'10.10.14.0/23': {'uphost_count': 1, 'responsive': {'10.10.14.16': ['ICMP']}},
 '192.168.3.0/24': {'uphost_count': 21,
  'responsive': {'192.168.3.1': ['ICMP'],
   '192.168.3.2': ['ICMP'],
   '192.168.3.21': ['ICMP'],
   '192.168.3.24': ['ICMP'],
   '192.168.3.27': ['ICMP'],
   '192.168.3.28': ['ICMP'],
   '192.168.3.29': ['ICMP'],
   '192.168.3.65': ['ICMP'],
   '192.168.3.95': ['ICMP'],
   '192.168.3.103': ['ICMP'],
   '192.168.3.113': ['ICMP'],
   '192.168.3.118': ['ICMP'],
   '192.168.3.121': ['ICMP'],
   '192.168.3.133': ['ICMP'],
   '192.168.3.138': ['ICMP'],
   '192.168.3.144': ['ICMP'],
   '192.168.3.146': ['ICMP'],
   '192.168.3.148': ['ICMP'],
   '192.168.3.195': ['ICMP'],
   '192.168.3.204': ['ICMP'],
   '192.168.3.216': ['ICMP']}},
 '192.168.20.0/30': {'uphost_count': 0, 'responsive': {}}}

In [42]:
    if tracking[range]['uphost_count'] == 0:
        unicorn_ports(range,top_20_ports,host_ips)

TOTAL Possible IPs: 2
TOTAL Possible IPs: 2
[*]RUNNING UNICORNSCAN -- TOP 20 PORTS on SAMPLE FROM 192.168.20.0/30[*]


In [43]:
    if tracking[range]['uphost_count'] == 0:
        nmap_top_hundred(range,host_ips)

[*]RUNNING NMAP TOP 100 PORTS -- SAMPLE FROM 192.168.20.0/30[*]


In [45]:
    df2 = pd.DataFrame.from_dict(tracking,orient='index')
    df2

,uphost_count,responsive
10.10.14.0/23,1,{'10.10.14.16': ['ICMP']}
192.168.3.0/24,21,"{'192.168.3.1': ['ICMP'], '192.168.3.2': ['ICM..."
192.168.20.0/30,0,{}


In [46]:
    print(tabulate(df2.sort_values(by='uphost_count',ascending=True),headers=('Uphost Count', 'Live Nodes'),tablefmt='grid'))
    #content2 =tabulate(df2.sort_values(by='uphost_count',ascending=False),headers=('Uphost Count','Live Nodes'),tablefmt='tsv')
    filename = 'logs/discovery-outfile-run-on-%s.csv'%datetime.datetime.now().strftime('%Y-%m-%d-%H%M')
    

+-----------------+----------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                 |   Uphost Count | Live Nodes                                                                                                                                                                                                                                                                                                                                                                     

In [17]:
    df2.to_csv(filename,index_label='IP Range')